In [1]:
import setup

setup.init_django()

In [2]:
from decouple import config, AutoConfig
config = AutoConfig(search_path="/home/harry/chatbotDjango") 

In [4]:
from data.models import BlogPost
from coffeshop.models import EmployeeRole, Employee, ProductType, Product, InventoryItem, ProductInventoryRequirement
from rag import db as rag_db, settings as rag_settings

In [5]:
from sqlalchemy import (
    create_engine,
    inspect,
)

from llama_index.core import SQLDatabase
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.retrievers import NLSQLRetriever

In [6]:
# initialize default LlamaIndex settings
rag_settings.init()
# get pooled Neon database string from .env or env vars
vector_database_url = rag_db.get_database_url(use_pooling=True)

In [7]:
engine = create_engine(vector_database_url)

In [8]:
inspect(engine).get_table_names()

['django_migrations',
 'django_content_type',
 'auth_permission',
 'auth_group',
 'auth_group_permissions',
 'auth_user_groups',
 'auth_user_user_permissions',
 'django_admin_log',
 'auth_user',
 'django_session',
 'products_embedding',
 'products_product',
 'data_blogpost',
 'analytics_pageview',
 'ragui_queryhistory',
 'coffeshop_employeerole',
 'coffeshop_employee',
 'coffeshop_producttype',
 'coffeshop_product',
 'coffeshop_inventoryitem',
 'coffeshop_productinventoryrequirement']

In [19]:
tables = []
models = [EmployeeRole, Employee, ProductType, Product, InventoryItem, ProductInventoryRequirement]
for model in models:
    table = model._meta.db_table
    tables.append(table)

In [20]:
tables

['coffeshop_employeerole',
 'coffeshop_employee',
 'coffeshop_producttype',
 'coffeshop_product',
 'coffeshop_inventoryitem',
 'coffeshop_productinventoryrequirement']

In [21]:
sql_database = SQLDatabase(engine, include_tables=tables)

In [22]:
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=tables,
)

In [23]:
response = sql_query_engine.query("What is my most recent blog post id?")
response

Response(response="It seems there was a misunderstanding. The database schema you're querying pertains to a coffee shop and includes information about employees, products, inventory, and roles. It does not contain any data related to blog posts. Therefore, I'm unable to retrieve information about your most recent blog post ID from this database. If you have any questions related to the coffee shop data, feel free to ask!", source_nodes=[NodeWithScore(node=TextNode(id_='885eee3a-5494-4485-9e04-66f1b1f41390', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Error: Statement "I\'m unable to answer the question as it is not related to the provided database schema. The schema includes information about a coffee shop, such as employees, products, inventory, and roles, but it does not include any information about blog posts. Therefore, I cannot generate a relevant 

In [24]:
for node in response.source_nodes:
    print(node.node.get_content())

Error: Statement "I'm unable to answer the question as it is not related to the provided database schema. The schema includes information about a coffee shop, such as employees, products, inventory, and roles, but it does not include any information about blog posts. Therefore, I cannot generate a relevant SQL query for this question." is invalid SQL.
Error: syntax error at or near "I"
LINE 1: I'm unable to answer the question as it is not related to th...
        ^



In [25]:
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=tables, return_raw=True
)

r = nl_sql_retriever.retrieve("What is americano?")

In [26]:
print(r)
for node in r:
    print(node)
    print(node.metadata)

[NodeWithScore(node=TextNode(id_='7cf8493a-bf71-491a-a102-2cd04cd3d291', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='Error: Statement \'To determine what an "americano" is, we need to check if it is a product type or a product itself. We\\\'ll start by querying the `coffeshop_product` table to see if there is a product named "americano."\\n\\n\\nSELECT coffeshop_product.name, coffeshop_product.description\\nFROM coffeshop_product\\nWHERE coffeshop_product.name = \\\'americano\\\';\' is invalid SQL.\nError: syntax error at or near "To"\nLINE 1: To determine what an "americano" is, we need to check if it ...\n        ^\n', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None)]
Node ID: 7cf8493a-bf71-491a-a102-2cd04cd3d291
Text: Error: Statement 'To determine what an